In [ ]:
print("toca chambear pepepeeee")

In [28]:
import json 
from collections import deque

class Nodo:
    def __init__(self, data=None):
        self.next = None
        self.data = data
        self.bef = None


class ColaDob:
    def __init__(self, limit=9):
        self.limite = limit
        self.front = None
        self.rear = None
        self.size = 0

    def isEmpty(self):
        return self.size <= 0

    def isFull(self):
        return self.size >= self.limite

    def enColarD(self, data):
        nodo =Nodo(data)
        if self.isFull():
            print("Cola llena")

        if self.isEmpty():
            self.front = self.rear = nodo
        else:
            nodo.bef= self.rear
            self.rear.next = nodo
            self.rear=nodo
        self.size += 1
        return data
    

    def desenColarD(self):
        if self.isEmpty():
            print("Cola vacía")
            return None
        nodo = self.front
        self.front = self.front.next
        if self.front is None:
            self.rear = None
        else:
            self.front.prev = None
        self.size -= 1
        return nodo.data

    def imprimirColaD(self):
        valor = self.front
        print("Cola: ", end="")
        while valor:
            print(valor.data, end=" => ")
            valor = valor.next
        print("None")


    def imprimirColaIz(self):
        valor = self.rear
        print("Cola: ", end="")
        while valor:
            print(valor.data, end=" => ")
            valor = valor.bef
        print("None")
    


    def captarColaS(self, ColaS):
        if ColaS.isEmpty():
            return
        else:
          self.enColarD(ColaS.desenColar())
          self.captarColaS(ColaS)


class PilaSAux:
    def __init__(self):
        self.top = None
        self.size = 0

    def isEmpty(self):
        return self.size <= 0

    def apilarAux(self, data):
        nodo = Nodo(data)
        if self.isEmpty():
            self.top = nodo
        else:
            nodo.next = self.top
            self.top = nodo
        self.size += 1
        return data

    def desenpilAux(self):
        if self.isEmpty():
            print("Pila vacía")
            return None
        elim = self.top.data
        self.top = self.top.next
        self.size -= 1
        return elim

    def captarColaD(self, ColaDob):
        if ColaDob.isEmpty():
            return
        else:
          self.apilarAux(ColaDob.desenColarD())
          self.captarColaD(ColaDob)

    def imprimirPilaAux(self):
        valor = self.top
        print("Pila Aux: ", end="")
        while valor:
            print(valor.data, end=" => ")
            valor = valor.next
        print("None")

In [24]:
cola = ColaDob()
cola.enColarD(1)
cola.enColarD(2)
cola.enColarD(3)

print('//////////////////')
cola.imprimirColaD()
print('//////////////////')
cola.imprimirColaIz()
print('//////////////////')
cola.desenColarD()
cola.imprimirColaD()


//////////////////
Cola: 1 => 2 => 3 => None
//////////////////
Cola: 3 => 2 => 1 => None
//////////////////
Cola: 2 => 3 => None


aca abajo la pila auxiliar que se podria usar para la cola 

In [ ]:
class PilaSAux:
    def __init__(self):
        self.top = None
        self.size = 0

    def isEmpty(self):
        return self.size <= 0

    def apilarAux(self, data):
        nodo = Nodo(data)
        if self.isEmpty():
            self.top = nodo
        else:
            nodo.next = self.top
            self.top = nodo
        self.size += 1
        return data

    def desenpilAux(self):
        if self.isEmpty():
            print("Pila vacía")
            return None
        elim = self.top.data
        self.top = self.top.next
        self.size -= 1
        return elim

    def captarColaD(self, ColaDob):
        if ColaDob.isEmpty():
            return
        else:
          self.apilarAux(ColaDob.desenColarD())
          self.captarColaD(ColaDob)

    def imprimirPilaAux(self):
        valor = self.top
        print("Pila Aux: ", end="")
        while valor:
            print(valor.data, end=" => ")
            valor = valor.next
        print("None")

//////////////////////////////////////////////////////////////////////////////////////////////////////////
Comienza lo buenos jsjsjjs

In [33]:

Cola = ColaDob()

with open('inventario1.txt', 'r') as file:
    for line in file:
        data = json.loads(line.strip())
        cola.enColarD(data)

# Imprimir la cola
cola.imprimirColaD()

FileNotFoundError: [Errno 2] No such file or directory: 'inventario1.txt'

In [34]:
import os
print("Directorio actual:", os.getcwd())

Directorio actual: c:\Users\USER\Documents\proyect\alg\Algoritmos-Trabajo-Final\backend


In [1]:
import json

class Nodo:
    def __init__(self, data=None):
        self.next = None
        self.data = data
        self.bef = None

class ColaDob:
    def __init__(self, limit=9):
        self.limite = limit
        self.front = None
        self.rear = None
        self.size = 0

    def isEmpty(self):
        return self.size <= 0

    def isFull(self):
        return self.size >= self.limite

    def enColarD(self, data):
        nodo = Nodo(data)
        if self.isFull():
            print("Cola llena")
            return

        if self.isEmpty():
            self.front = self.rear = nodo
        else:
            nodo.bef = self.rear
            self.rear.next = nodo
            self.rear = nodo
        self.size += 1
        return data

    def desenColarD(self):
        if self.isEmpty():
            print("Cola vacía")
            return None
        nodo = self.front
        self.front = self.front.next
        if self.front is None:
            self.rear = None
        else:
            self.front.bef = None
        self.size -= 1
        return nodo.data

    def imprimirColaD(self):
        valor = self.front
        print("Cola: ", end="")
        while valor:
            print(valor.data, end=" => ")
            valor = valor.next
        print("None")

    def imprimirColaIz(self):
        valor = self.rear
        print("Cola: ", end="")
        while valor:
            print(valor.data, end=" => ")
            valor = valor.bef
        print("None")

    def captarColaS(self, ColaS):
        if ColaS.isEmpty():
            return
        else:
            self.enColarD(ColaS.desenColar())
            self.captarColaS(ColaS)

# Crear una instancia de la cola doblemente enlazada
cola = ColaDob()

# Leer el archivo de texto y llenar la cola
try:
    with open('inventario1.txt', 'r') as file:
        for line in file:
            data = json.loads(line.strip())
            cola.enColarD(data)
except FileNotFoundError:
    print("El archivo 'inventario1.txt' no se encontró. Asegúrate de que el archivo esté en el mismo directorio que tu código.")

# Imprimir la cola
cola.imprimirColaD()

print('///////////////')
cola.imprimirColaIz()


Cola: {'id': 1, 'nombre': 'Manzana', 'tipo': 'perecedero', 'categoria': 'fruta', 'cantidad': 50, 'fech_venc': '2024-10-26'} => {'id': 2, 'nombre': 'Arroz', 'tipo': 'no perecedero', 'categoria': 'grano', 'cantidad': 100, 'fech_venc': '2030-01-20'} => {'id': 3, 'nombre': 'Leche', 'tipo': 'perecedero', 'categoria': 'lacteo', 'cantidad': 30, 'fech_venc': '2025-11-10'} => {'id': 4, 'nombre': 'Lentejas', 'tipo': 'no perecedero', 'categoria': 'grano', 'cantidad': 80, 'fech_venc': '2024-10-26'} => {'id': 5, 'nombre': 'Pollo', 'tipo': 'perecedero', 'categoria': 'carnes', 'cantidad': 20, 'fech_venc': '2024-10-27'} => {'id': 6, 'nombre': 'Pasta', 'tipo': 'no perecedero', 'categoria': 'pasta', 'cantidad': 60, 'fech_venc': '2027-12-12'} => None
///////////////
Cola: {'id': 6, 'nombre': 'Pasta', 'tipo': 'no perecedero', 'categoria': 'pasta', 'cantidad': 60, 'fech_venc': '2027-12-12'} => {'id': 5, 'nombre': 'Pollo', 'tipo': 'perecedero', 'categoria': 'carnes', 'cantidad': 20, 'fech_venc': '2024-10-27

In [2]:
import tkinter as tk
from tkinter import messagebox
import json

class InventarioApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Gestión de Inventarios")
        
        # Campos de entrada
        self.id_entry = self.create_entry("ID", 0)
        self.nombre_entry = self.create_entry("Nombre", 1)
        self.tipo_entry = self.create_entry("Tipo", 2)
        self.categoria_entry = self.create_entry("Categoría", 3)
        self.cantidad_entry = self.create_entry("Cantidad", 4)
        self.fech_venc_entry = self.create_entry("Fecha de Vencimiento", 5)
        
        # Botones
        self.add_button = tk.Button(root, text="Agregar", command=self.agregar_item)
        self.add_button.grid(row=6, column=0, pady=10)
        
        self.search_button = tk.Button(root, text="Buscar", command=self.buscar_item)
        self.search_button.grid(row=6, column=1, pady=10)
        
        # Lista para mostrar los datos
        self.listbox = tk.Listbox(root, width=80)
        self.listbox.grid(row=7, column=0, columnspan=2, pady=10)
        
        # Cargar datos desde el archivo
        self.datos = []
        self.cargar_datos()

    def create_entry(self, label_text, row):
        label = tk.Label(self.root, text=label_text)
        label.grid(row=row, column=0, padx=10, pady=5)
        entry = tk.Entry(self.root)
        entry.grid(row=row, column=1, padx=10, pady=5)
        return entry

    def agregar_item(self):
        item = {
            "id": self.id_entry.get(),
            "nombre": self.nombre_entry.get(),
            "tipo": self.tipo_entry.get(),
            "categoria": self.categoria_entry.get(),
            "cantidad": self.cantidad_entry.get(),
            "fech_venc": self.fech_venc_entry.get()
        }
        self.datos.append(item)
        self.listbox.insert(tk.END, item)
        self.guardar_datos()
        messagebox.showinfo("Éxito", "Item agregado exitosamente")

    def buscar_item(self):
        nombre = self.nombre_entry.get()
        resultados = [item for item in self.datos if item["nombre"] == nombre]
        self.listbox.delete(0, tk.END)
        for item in resultados:
            self.listbox.insert(tk.END, item)

    def cargar_datos(self):
        try:
            with open('inventario1.txt', 'r') as file:
                for line in file:
                    data = json.loads(line.strip())
                    self.datos.append(data)
                    self.listbox.insert(tk.END, data)
        except FileNotFoundError:
            print("El archivo 'inventario1.txt' no se encontró.")

    def guardar_datos(self):
        with open('inventario1.txt', 'w') as file:
            for item in self.datos:
                file.write(json.dumps(item) + '\n')

if __name__ == "__main__":
    root = tk.Tk()
    app = InventarioApp(root)
    root.mainloop()
